In [1]:
from selenium import webdriver # navegador web automatizado
from time import sleep

from util import testa_arquivo, criar_arquivo, leia_numero_processo,valida_processo, elementoExiste, escreve_arquivo

if testa_arquivo():
    print('Acessando arquivo de configuração: OK')
    import variaveis
    texto = variaveis.texto

try:
    painel_do_servidor = variaveis.painel_do_servidor
    print('Caminho para o Painel do Servidor encontrado.', painel_do_servidor)
except:
    painel_do_servidor = False
    print('Caminho para o Painel do Servidor não encontrado. Não se preocupe!')
espera = 1

print('Inicializando o navegador autmático.')
#inicializa o navegador
driver = webdriver.Chrome('./chromedriver.exe')

if painel_do_servidor == False:
    print('Percorrendo o site em busca do PAINEL DO SERVIDOR')
    driver.get("http://sistemasnet/sarh/") # acessa o SARH
    sleep(10) # espera aguardar para colocar a senha, se não estiver usando computador corporativo.

    driver.find_element_by_link_text("Recursos Humanos (SARH)").click()
    sleep(espera) # aguarda carregar
    driver.find_element_by_link_text("HORÁRIO FLEXÍVEL").click()
    sleep(espera) # aguarda carregar
    driver.find_element_by_link_text("PAINEL DO SERVIDOR").click()
    sleep(espera) # aguarda carregar
    # pega a url do painel do servidod
    # todo: salvar no arquivo de variáveis o painel do servidor
    painel_do_servidor = driver.current_url
    print('PAINEL DO SERVIDOR encontrado!')
    # adiciona a 
    escreve_arquivo(url=painel_do_servidor, filePath='variaveis.py')
    print('PAINEL DO SERVIDOR adicionado ao arquivo de variáveis')

else:
     driver.get(painel_do_servidor) 
    
# listar os dias editaveis do mês corrente
## percorre todas as linhas procurando as que não foram preenchidas
html = driver.find_element_by_css_selector('#tableEvento').get_attribute('innerHTML')
html = html.split('>07-006<')
html.remove(html[0])

url_tela_de_controle = "http://sistemasnet/sarh/HorarioFlexivelNovo/Gerente/Controle/"

for i,v in enumerate(html):
    print(f'Editando dia {i+1} de {len(html)}')
    inicio_link = v.find('CadastrarOcorrencia.asp')
    fim_link = v.find('=">Editar')
    link = v[inicio_link:fim_link].replace('amp;','')
    
    driver.get(url_tela_de_controle+link)

    fonte = driver.page_source
    posicao = fonte.find('<div style="display:inline;" id="divSaldo"><font color="red">-')
    # saldo do dia vai preencher o campo de horas de Evento Externo (id=TempoJ13)
    saldo = fonte[posicao+62:posicao+67]
    #print(saldo)

    # todo: testar o formato do saldo

    sleep(1)
    #print(f's= {saldo}')
    # preenche o saldo no campo 07-013 Evento Externo
    script  = f'document.getElementById(\'TempoJ13\').value=\'{saldo}\'; \n'
    script += 'valida_horas2(document.getElementById(\'TempoJ13\'),\'TempoJ13\'); \n'
    script += 'RecalculaJornada(13,document.getElementById(\'TempoJ13\').value); \n'
    script += f'document.getElementById(\'descJustificativaJ13\').value = \'{texto}\'; \n'
    script += 'document.getElementById(\'Tempo6\').value=\'00:00\';'
    script += 'valida_horas2(document.getElementById(\'Tempo6\'),\'Tempo6\');'
    script += 'RecalculaSaldoTotal(6,document.getElementById(\'Tempo6\').value);'
    script += 'salvar();'
    driver.execute_script(script)

    sleep(espera)

    alert = driver.switch_to.alert
    alert.accept()
        
print('Não existem dias pendentes no mês corrente.')
driver.get(painel_do_servidor) # ACESSA o Painel do servidor
sleep(espera)

# colocar o código do mês anterior
print('Verificando saldo do mês anterior.')


Acessando arquivo de configuração: OK
Caminho para o Painel do Servidor encontrado. http://sistemasnet/sarh/horarioflexivelNovo/gerente/controle/tela.asp?SISQSmodulo=18421
Inicializando o navegador autmático.
Editando dia 1 de 5
Editando dia 2 de 5
Editando dia 3 de 5
Editando dia 4 de 5
Editando dia 5 de 5
Não existem dias pendentes no mês corrente.
Verificando saldo do mês anterior.
